### Explore the Dataset 

In [2]:
from IPython.display import display
import pandas as pd
# Define the path to your local file
local_file_path = "./data/energy_data_correct.csv"

# Read the local file into a pandas DataFrame
df = pd.read_csv(local_file_path)


print("Sample of Dataset")
display(df.head())

print("\n Describe Dataset")
display(df.describe())

Sample of Dataset


,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28



 Describe Dataset


,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307195,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090204,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


### Upload and Register the dataset into Azure


In [2]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes,InputOutputModes
from azure.identity import DefaultAzureCredential
from azure.ai.ml import Input

# Connect to the AzureML workspace
subscription_id="<YOUR subscription_id>"
resource_group="<YOUR resource_group>"
workspace_name="<YOUR workspace_name>"


ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

# Set the version number of the data asset (for example: '1')
VERSION = "1"

# Set the path, supported paths include:
# local: './<path>/<file>' (this will be automatically uploaded to cloud storage)
# blob:  'wasbs://<container_name>@<account_name>.blob.core.windows.net/<path>/<file>'
# ADLS gen2: 'abfss://<file_system>@<account_name>.dfs.core.windows.net/<path>/<file>'
# Datastore: 'azureml://datastores/<data_store_name>/paths/<path>/<file>'


#local path
path = "./data/energy_data_correct.csv"

# Define the Data asset object
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="this is an energy building dataset, I am making a test here",
    name="energy-building-for-ml",
    version=VERSION,
)

# Create the data asset in the workspace
ml_client.data.create_or_update(my_data)


Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'energy-building-for-ml', 'description': 'this is an energy building dataset, I am making a test here', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/dd022f57-1b53-4cf0-b379-44a3d7d57e27/resourceGroups/ies-pi-dev-uks-rg/providers/Microsoft.MachineLearningServices/workspaces/ies-pi-dev-uks-ml/data/energy-building-for-ml/versions/2', 'Resource__source_path': None, 'base_path': 'c:\\Users\\saeed.misaghian\\Documents\\Repos_Personal\\Azure_ML_Energy_Predict', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000024818CCED00>, 'serialize': <msrest.serialization.Serializer object at 0x0000024819281F70>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/dd022f57-1b53-4cf0-b379-44a3d7d57e27/resourcegroups/ies-pi-dev-uks

### Call the training script and log the metrics

In [4]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes,InputOutputModes
from azure.identity import DefaultAzureCredential
from azure.ai.ml import Input
from azure.ai.ml import command


# get dataset id 
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("energy-building-for-ml", version="1")


job = command(
    code="./src",
    command="python train-model-script.py --training_data ${{inputs.training_data}}",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="building-energy-train",
    experiment_name="building-energy-training", 
    tags={"model_type": "randomforest", "model": "Building energy dataset"},
    inputs={'training_data': Input(path=data_asset.id,
              type=AssetTypes.URI_FILE,
              mode=InputOutputModes.RO_MOUNT)}  # Reference the dataset by name
)

# Submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Found the config file in: .\config.json
Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information

Monitor your job at https://ml.azure.com/runs/helpful_roti_6611tqzhfr?wsid=/subscriptions/dd022f57-1b53-4cf0-b379-44a3d7d57e27/resourcegroups/ies-pi-dev-uks-rg/workspaces/ies-pi-dev-uks-ml&tid=b33be5d6-5072-448f-bad3-d8b66cf09736


### Create Script

- In case, you want to create the folder and script from notebook:

In [5]:
import os

# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


In [ ]:
%%writefile $script_folder/train-model-script.py

#write the script code here: 


In [1]:
ws

NameError: name 'ws' is not defined

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes,InputOutputModes
from azure.identity import DefaultAzureCredential
from azure.ai.ml import Input
from azure.ai.ml import command
from azureml.core import  Environment


# Connect to the AzureML workspace
subscription_id="dd022f57-1b53-4cf0-b379-44a3d7d57e27"
resource_group="ies-pi-dev-uks-rg"
workspace_name="ies-pi-dev-uks-ml"


# get dataset id 
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("energy-building-for-ml", version="1")

custom_env = Environment.from_conda_specification(name="myenv", file_path="./conda_env.yml")
custom_env.register(workspace=workspace_name)

job = command(
    code="./src",
    command="python train-model-mlflow.py --training_data ${{inputs.training_data}}",
    environment="myenv",
    compute="aml-cluster",
    display_name="building-energy-train",
    experiment_name="building-energy-training", 
    tags={"model_type": "randomforest", "model": "Building energy dataset"},
    inputs={'training_data': Input(path=data_asset.id,
              type=AssetTypes.URI_FILE,
              mode=InputOutputModes.RO_MOUNT)}  # Reference the dataset by name
)

# Submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In [3]:
import sklearn
import xgboost
import argparse
import numpy
import pandas

# print("Python version:", platform.python_version())
print("scikit-learn version:", sklearn.__version__)
print("xgboost version:", xgboost.__version__)
print("argparse is a standard library, its version is tied with Python's version.")
print("numpy version:", numpy.__version__)
print("pandas version:", pandas.__version__)


scikit-learn version: 1.2.2
xgboost version: 1.7.3
argparse is a standard library, its version is tied with Python's version.
numpy version: 1.23.5
pandas version: 1.5.3
